In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from urllib.parse import quote, unquote
import time

In [36]:
def get_info_from_data_portal(**kwargs):
    url = 'https://data.icos-cp.eu/portal/#' + str(kwargs).replace("'", '"')
    url = quote(url, safe=':/#')
    
    #decoded_url = unquote(url)
    #options = webdriver.ChromeOptions()
    #options.add_argument('--headless=new')

    # Set up Selenium WebDriver (e.g., Chrome)
    count = 0
    scraped = []

    with webdriver.Chrome() as driver:  # Ensure you have the ChromeDriver installed
        while count < 10:
            # Load the page
            #tries next:
            #url = None
            if url == None: break
            
            driver.get(url)
            # Wait for the page to fully render (optional but recommended)
            WebDriverWait(driver, 10).until(  # Wait up to 10 seconds
                EC.presence_of_element_located((By.ID, "cp-site-footer")))
            time.sleep(3)

            content = BeautifulSoup(driver.page_source, 'html.parser').find('div')

            for c in range(count):
                span = driver.find_element(By.CSS_SELECTOR, "span.fas.fa-step-forward")
                spanParent = span.find_element(By.XPATH, "..")
                get_style = lambda webelem: dict([el.split(':') for el in webelem.get_attribute("style").replace(' ', '').split(';')  if ':' in el])
                
                if span and span.is_enabled() and get_style(spanParent).get('opacity', False)==False:
                    print('Check '+ 'next'*count + '.'*c)
                    span.click()
                    time.sleep(3)
                else:
                    return scraped

            # Parse the HTML
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all <a> tags with the title "View metadata"
            links = soup.find_all('a', title="View metadata")

            # Extract href attributes
            hrefs = [link['href'] for link in links]
            sids = [h.rsplit('/', 1)[-1] for h in hrefs]

            for sid, href in list(zip(sids, hrefs)):
                row_data = {"sid": sid}

                driver.get(href)
                WebDriverWait(driver, 10).until(  # Wait up to 10 seconds
                    EC.presence_of_element_located((By.ID, "meta-add-to-cart-button")))
                time.sleep(3)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                source = soup.find('label', string="Citation").parent.find_next_sibling().text.strip()
                site_meta = soup.find('label', string="Station").parent.find_next_sibling().find('a')['href']
                
                row_data.update({"Source": source})
                
                driver.get('https://meta.icos-cp.eu' + site_meta)
                WebDriverWait(driver, 10).until(  # Wait up to 10 seconds
                    EC.presence_of_element_located((By.ID, "cp-site-footer")))
                time.sleep(3)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                station_id = soup.find('label', string="Station ID").parent.find_next_sibling().text.strip()
                latlon = soup.find('label', string="Latitude/Longitude").parent.find_next_sibling().text.strip()
                latlon = tuple([float(el) for el in latlon.replace(' ', '').split(',')])

                row_data.update({"id": station_id, "latlon": latlon})

                driver.get('https://meta.icos-cp.eu/labeling/')
                WebDriverWait(driver, 10).until(  # Wait up to 10 seconds
                    EC.presence_of_element_located((By.ID, "cp-site-footer")))
                time.sleep(3)
                parent_element = driver.find_element(By.XPATH, "//div[@data-reactid='.0.3']")

                possible_names = {station_id: [station_id, station_id.upper()]}
                possible_names[station_id] += {'FR-Lqu': ['FR-LQ1'], 'FI-Fos': ['FI-TVA'], 'FI-Hyy': ['ECO-HYY'], 'CH-Dav':['DAV'], 'SE-Htm': ['SE-Hyl'],
                                               'FR-FBn': ['FR-BLA'], 'FI-Sod': ['ECO-SOD'], 'FI-Sii': ['ECO-Sii'], 'SE-Svb': ['SE-Sva'], 'GL-ZaF': ['DK-ZaF']}.get(station_id, [])

                for codenames in possible_names.get(station_id, []):
                    base_reactid = f'.0.3.1:$http=2//meta=1icos-cp=1eu/ontologies/stationentry/ES/{codenames}'
                    sections = parent_element.find_elements(By.XPATH, f".//div[@data-reactid='{base_reactid}.0.0']")
                    if len(sections) > 0: break
                    base_reactid = f'.0.3.1:$http=2//meta=1icos-cp=1eu/resources/stationentry/{codenames}'
                    sections = parent_element.find_elements(By.XPATH, f".//div[@data-reactid='{base_reactid}.0.0']")
                    if len(sections) > 0: break

                for site in sections:
                    sitecode = site.get_attribute('data-reactid').rsplit('/', 1)[-1].split('.', 1)[0]
                    # Use ActionChains to click the element
                    actions = ActionChains(driver)
                    actions.move_to_element(site).click().perform()
                    # Wait for the new element to appear after the click
                    properties = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, f".//form[@data-reactid='{base_reactid}.0.0.1.0.1.1.0']"))
                    )
                    rows = properties.find_elements(By.XPATH, ".//div[contains(@class, 'row')]")

                    full_name = site.find_element(By.XPATH, f".//span[@data-reactid='{base_reactid}.0.0.0.1']").text
                    row_data.update({"Full name": full_name})

                    for row in rows:
                        if row == None or row.find_elements(By.TAG_NAME, "label") == None: continue
                        label = row.find_element(By.TAG_NAME, "label").text  # Get the label text
                        value = row.find_element(By.TAG_NAME, "input").get_attribute('value')
                        #if input_tag:
                        #    value = input_tag.text #or str(input_tag.get("checked", False))  # Handle text or checkbox
                        #else:
                        #    value = None
                        row_data.update({label: value})
                
                #print(row_data)
                scraped += [row_data]
            
            count +=1        

    return scraped


In [37]:
# { "filterCategories": { "project": ["icos"], "level": [2], "stationclass": ["ICOS"], "type": ["etcL2Fluxes"], "station": ["iES_BE-Bra"] }}
cp_etc_l2_flux = get_info_from_data_portal(**{ "filterCategories": { "project": ["icos"], "level": [2], "stationclass": ["ICOS"], "type": ["etcL2Fluxes"] },"tabs":{"resultTab":1}})
pd.DataFrame(cp_etc_l2_flux).to_csv('sample/output/cp_etc_l2_flux_info_lastrun.csv', index=False)
cp_etc_l2_flux


Check next
Check nextnext


[{'sid': 'AzoBa8mYF6-Rg5wCsdewWOPN',
  'Source': 'Ibrom, A., Lange Rønn, E., Møller, F., Pilegaard, K., Schaarup Sørensen, J. (2024). ETC L2 Fluxes, Soroe, 2020-12-31–2024-10-20, ICOS RI, https://hdl.handle.net/11676/AzoBa8mYF6-Rg5wCsdewWOPN',
  'id': 'DK-Sor',
  'latlon': (55.48587, 11.644645),
  'Full name': 'Soroe',
  'Latitude [WGS84, decimal degrees with 5 decimals]': '55.48587',
  'Longitude [WGS84, decimal degrees with 5 decimals]': '11.64464',
  'Station class': '1',
  'Anemometer arm direction, degrees from north': '220',
  'Height of the eddy covariance system, meters above ground': '43',
  'Submitted sufficient wind data to the European Database': 'on'},
 {'sid': '_7Cqp56ptTcWxvq3H6BkBJ0d',
  'Source': 'Rebmann, C., Dienstbach, L., Schmidt, P., Wiesen, R., Meis, J., Feldmann, I., Bastos Campos, F., Garcia Quiros, I., Gimper, S., Hildebrandt, A., Paasch, S. (2024). ETC L2 Fluxes, Hohes Holz, 2018-12-31–2024-10-20, ICOS RI, https://hdl.handle.net/11676/_7Cqp56ptTcWxvq3H6BkBJ0d

In [38]:
import numpy as np
import pandas as pd
buffer = lambda zm: np.ceil((np.log(zm) * 500) / 2 / 100) * 100

# Print zm to buffer curve
#import matplotlib.pyplot as plt
#plt.plot(np.linspace(1, 100, 100), [buffer(z) for z in np.linspace(1, 100, 100)])
#print(buffer(2.6), buffer(37))

cp_etc_l2_flux_data = pd.read_csv('sample/output/cp_etc_l2_flux_info_lastrun.csv')
for _, row in cp_etc_l2_flux_data.iterrows():
    if np.isnan(row['Height of the eddy covariance system, meters above ground']):
        print('|' + row.id + '|', row['Full name'])
    #print(row['id'], buffer(row['Height of the eddy covariance system, meters above ground']))


|DK-Vng| nan
